Subsetting ERA5 to political boundaries
==================

In this notebook, we download ERA5 precipitation and temperature data and subset it to political boundaries (countries and states/provinces). We write these out to two csv files.

In [ ]:
%matplotlib inline

import warnings

import cartopy
import cdsapi
import geopandas
import hvplot.pandas  # noqa
import regionmask

import pandas as pd
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
from cartopy import crs as ccrs
from tqdm.notebook import tqdm

xr.set_options(display_style='html')
warnings.simplefilter("ignore", category=RuntimeWarning)
assert regionmask.__version__ == '0.5.0'

## Download the latest ERA5 data

These requires authenticating with the Copernicus Climate Data Store. 

You'll need a file named `.cdsapirc` in your home directory with the following details:
```
url: https://cds.climate.copernicus.eu/api/v2
key: XXXX:1234567-1234-1234-1234-1234567890
```
Be sure to replace both parts of the `XXXX` with your UID and the remainder with your key.

In [ ]:
# url: https://cds.climate.copernicus.eu/api/v2
# key: XXXX:1234567-1234-1234-1234-1234567890
c = cdsapi.Client()

This next cell will download data from the CDS, placing a new file (`download.nc`) in your working directory. 

In [ ]:
c.retrieve(
    'reanalysis-era5-single-levels-monthly-means',
    {
        'format': 'netcdf',
        'product_type': 'monthly_averaged_reanalysis',
        'variable': [
            '2m_temperature', 'total_precipitation',
        ],
        'year': [
            '2019', '2020',
        ],
        'month': [
            '01', '02', '03',
            '04', '05', '06',
            '07', '08', '09',
            '10', '11', '12',
        ],
        'time': '00:00',
    },
    'download.nc')

Now we can open the ERA5 dataset. This dataset comes with two experiements, we'll merge them since they don't overlap in their forecast time.

In [ ]:
# open the dataset
ds = xr.open_dataset('download.nc')
# merge the experiments
ds = ds.bfill('expver').isel(expver=0)
ds

In [ ]:
# plot the last timestep
ds['t2m'].isel(time=-1).plot()

In [ ]:
# plot a timeseries at one location
ds['t2m'].sel(longitude=114.3055, latitude=30.5928, method='nearest').plot()

## Subsetting ERA5 by political boundaries

The ERA5 data we plotted above is still in its gridded format. If we want to subset this data by political boundary, we need to bring in a shapefile. Below we define two functions that will help us do this subsetting.

In [ ]:
def get_gdf(resolution='50m', category='cultural', name='admin_0_countries'):
    '''return a geopandas.GeoDataFrame of boundaries
    
    More info: https://www.naturalearthdata.com/downloads/
    '''
    fname = cartopy.io.shapereader.natural_earth(
      resolution=resolution,
      category=category,
      name=name)    

    gdf = geopandas.GeoDataFrame.from_file(fname)   
    gdf['cent_lon'] = gdf.geometry.centroid.x
    gdf['cent_lon'].values[gdf['cent_lon'].values < 0] += 360.
    gdf['cent_lat'] = gdf.geometry.centroid.y
    
    return gdf


def subset_by_gdf(ds, gdf, var='t2m'):
    '''Subset a dataset (ds) using the shapes defined in a GeoDataFrame (gdf)
    
    Returns
    -------
    final_df : geopandas.GeoDataFrame
    
    '''
    # create masks
    print('Creating masks...')
    shapes = regionmask.Regions(gdf.geometry)
    mask = shapes.mask(ds, lon_name='longitude', lat_name='latitude', wrap_lon=True)

    print('looping over shapes...')
    # loop over shapes
    df = pd.DataFrame(index=ds.indexes['time'])
    for val, row in tqdm(gdf.iterrows()):
        if not (mask == val).values.any():
            data = ds[var].sel(latitude=row['cent_lat'], longitude=row['cent_lon'], method='nearest', tolerance=1)
        else:
            data = ds[var].where(mask == val).mean(('latitude', 'longitude'))
        df[val] = data.to_series()
        if var == 't2m':
            df[val] -= 273.15
    
    # setup final dataframe
    df.index = df.index.to_period()
    df = df['2019-11-01':].transpose()
    
    final_df = gdf.merge(df, right_index=True, left_index=True)
    final_df.crs = ccrs.PlateCarree().proj4_init
    
    return final_df

In [ ]:
gdf = get_gdf(resolution='110m')
countries = subset_by_gdf(ds, gdf, var='t2m')

display(countries.head())
countries.hvplot(c='2019-12', cmap='viridis', hover_cols=['ADMIN'])

In [ ]:
gdf = get_gdf(resolution='10m', name='admin_1_states_provinces')
states = subset_by_gdf(ds, gdf, var='t2m')

display(states.head())
states.hvplot(c='2019-12', cmap='viridis')  # TODO fix the hover tools here

In [ ]:
# write to csv
pd.DataFrame(countries).drop(columns=['geometry']).to_csv('era5_ne_countries.csv', encoding='utf-8')
pd.DataFrame(states).drop(columns=['geometry']).to_csv('era5_ne_states.csv', encoding='utf-8')